In [1]:
using PauliPropagation
using Base.Threads
using Plots

In [2]:
tn_extrapolated = [9.99999254e-01,  9.99593653e-01,  9.95720077e-01,  9.88301532e-01,
        9.78553511e-01,  9.58023054e-01,  9.21986059e-01,  8.81726079e-01,
        8.49816779e-01,  8.24900527e-01,  7.91257641e-01,  7.37435202e-01,
        6.68573798e-01,  5.88096040e-01,  4.81874079e-01,  3.50316579e-01,
        2.26709331e-01,  1.39724659e-01,  7.86639143e-02,  4.24124371e-02,
        1.90595136e-02,  6.18879050e-03, -8.27168956e-04, -4.63372099e-03,
       -7.05202121e-03, -7.68387421e-03, -6.33121142e-03, -4.32594440e-03,
        6.52050191e-04,  1.72598340e-04,  5.64696020e-05, -7.70582375e-07]

tn_angles = LinRange(0, π/2, length(tn_extrapolated));

In [3]:
function kickedisingcircuit(nq, nl; topology=nothing)
    
    # just in case a topology is not provided
    if isnothing(topology)
        topology = bricklayertopology(nq)
    end
    
    # define a layer of parametrized Rx-gates
    xlayer(circuit) = append!(circuit, (PauliGate([:X], [qind]) for qind in 1:nq))
    
    # define a layer of fixed ZZ(π/2)-gates
    zzlayer(circuit) = append!(circuit, (PauliGate([:Z, :Z], pair, π/2) for pair in topology))
    
    # define the empty circuit
    circuit = Gate[]
    # append to the circuit
    for _ in 1:nl
        zzlayer(circuit)
        xlayer(circuit)
    end
    
    return circuit
end

kickedisingcircuit (generic function with 1 method)

In [4]:
# number of qubits
nq = 127
# the IBM Eagle topology
topology = ibmeagletopology
# 20 layers for the hardest simulation
nl = 20
# create the circuit
const circuit = kickedisingcircuit(nq, nl; topology);
# count the number of parameters
const nparams = countparameters(circuit)
# define the observable (note the index is 62 if you start with 0)
const pstr = PauliString(nq, :Z, 63)

PauliString(nqubits: 127, 1.0 * IIIIIIIIIIIIIIIIIIII...)

In [5]:
# perform the simulation for one x-angle
function onesimulation(angle; min_abs_coeff=0, max_weight=Inf)
    # set the angles of the Rx gates
    thetas = ones(nparams) * angle
    # propagate
    psum = propagate(circuit, pstr, thetas; min_abs_coeff, max_weight)
    # overlap with the zero-state
    return overlapwithzero(psum)
end

onesimulation (generic function with 1 method)

In [ ]:
# set the truncations
min_abs_coeff = 3e-5
max_weight = 10

# prepare everything and run
x_angles = LinRange(0, π/2, 20)
expectations = zeros(length(x_angles))
t = @timed @threads for ii in eachindex(x_angles)
    angle = x_angles[ii]
    @time expectations[ii] = onesimulation(angle; min_abs_coeff, max_weight)
end

print("Total simulation time: ", t.time, " seconds")

  0.443960 seconds (3.34 M allocations: 130.168 MiB, 463.75% compilation time)
  0.219989 seconds (3.24 M allocations: 123.933 MiB, 730.08% compilation time)
  0.212681 seconds (6.52 M allocations: 217.457 MiB, 67.74% gc time, 42.88% compilation time)
  0.168639 seconds (2.18 M allocations: 76.357 MiB, 85.44% gc time)
  0.793544 seconds (19.66 M allocations: 666.539 MiB, 18.16% gc time, 263.57% compilation time)
  0.586036 seconds (20.59 M allocations: 699.059 MiB, 24.59% gc time, 318.70% compilation time)
  1.556342 seconds (73.54 M allocations: 2.194 GiB, 24.09% gc time, 105.62% compilation time)
  2.515060 seconds (95.23 M allocations: 2.685 GiB, 48.04% gc time)
  7.908962 seconds (304.22 M allocations: 8.566 GiB, 32.09% gc time, 26.44% compilation time)
 16.927882 seconds (636.49 M allocations: 18.015 GiB, 35.20% gc time, 0.54% compilation time)
 18.128618 seconds (609.86 M allocations: 17.225 GiB, 38.19% gc time)
  8.929862 seconds (289.13 M allocations: 8.050 GiB, 39.23% gc time)

In [ ]:
plot(tn_angles, tn_vals, label="Tensor network converged", color="black", linewidth=3)
plot!(x_angles, expectations, marker=:circle, label="PP Max weight = $max_weight, Min coeff = $min_abs_coeff", linewidth=3, color="royalblue")
